# Cell 1: 安装（必须先运行，transformers 4.44 装到独立目录，避免与系统 5.x 冲突）

# Cell 0: 环境设置（Kaggle P100 / Colab）

- **P100 兼容**：Kaggle 预装 PyTorch cu128 不支持 P100 (sm_60)，需先运行下方安装 Cell 安装 PyTorch cu118
- **安装后**：若仍报 GPU 不兼容，请 **Restart Session** 后重新 Run All
- P100 不支持 BF16，自动用 FP16
- **pip 的 dependency conflicts 警告可忽略**：transformers 装在独立目录，不影响系统包

In [ ]:
# 安装 transformers 4.44 到独立目录，避免与系统 5.x 冲突（不降级 numpy/pandas，减少依赖冲突）
import sys
import os
TF_ENV = "/content/tf_env" if os.path.exists("/content") else "/kaggle/working/tf_env"
# 移除可能残留的旧路径，确保加载本次安装
for p in ["/kaggle/working/transformers_4.46", "/content/transformers_4.46"]:
    if p in sys.path:
        sys.path.remove(p)
# 先安装 torch+torchvision 到 TF_ENV，确保版本匹配（避免 torchvision::nms 报错）
if os.path.exists("/kaggle"):
    !pip install --target $TF_ENV torch==2.7.1 torchvision==0.22.1 torchaudio==2.7.1 --index-url https://download.pytorch.org/whl/cu118 -q 2>&1 | grep -vE "dependency conflicts|incompatible" || true
else:
    !pip install --target $TF_ENV torch==2.7.1 torchvision==0.22.1 -q 2>&1 | grep -vE "dependency conflicts|incompatible" || true
# 分两步安装：先 tokenizers 0.20，再 transformers（避免 pip 依赖冲突）
!pip install --target $TF_ENV --no-cache-dir -q "tokenizers>=0.20,<0.21" 2>&1 | grep -vE "dependency conflicts|incompatible" || true
!pip install --target $TF_ENV --no-cache-dir -q transformers==4.44.2 radgraph 2>&1 | grep -vE "dependency conflicts|incompatible" || true
!pip install -q pillow jinja2 2>&1 | grep -vE "dependency conflicts|incompatible" || true

# 必须优先加载 TF_ENV，否则会用到系统 transformers
sys.path.insert(0, TF_ENV)
for k in list(sys.modules.keys()):
    if k == "transformers" or k.startswith("transformers."):
        del sys.modules[k]
import transformers
print(f"✓ transformers: {transformers.__version__} from {transformers.__file__}")
print("✅ 安装完成！")

# Cell 1: 原始 MedGemma (W4A16/FP16) + F1 + RadGraph

## 说明
- **W4A16**：4-bit 权重（此处为基线，实际为 FP16/BF16 全精度）+ 16-bit 激活
- 本 Notebook 为**原始模型**基线，用于与 W4A4、W4A8 量化版本对比

## Kaggle 输入
- **Add Input** 添加 `mimic-cxr-dataset`（含 official_data_iccv_final/files/ 图片）
- **mimic_eval_single_image_final_233.csv**：可单独建数据集上传，或放入 mimic-cxr-dataset 根目录

# Cell 2.5: HuggingFace 登录（MedGemma 1.5 为 gated 模型，需授权）

- 在 [模型页](https://huggingface.co/google/medgemma-1.5-4b-it) 申请访问
- Kaggle：Add-ons → Secrets 添加 secret，Label 填 `zhuxiruimedgamma`，Value 填你的 HuggingFace token

In [ ]:
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
# Kaggle Secrets 中 secret 名称需与下方一致（如 zhuxiruimedgamma 存的是 HF token）
user_secrets = UserSecretsClient()
tok = user_secrets.get_secret("zhuxiruimedgamma")
if tok:
    login(token=tok)
else:
    print("未找到 secret，请确认 Kaggle Add-ons → Secrets 已添加 zhuxiruimedgamma（值为 HF token）")

# 跳过 pipeline demo（Kaggle 环境有 numpy/scipy 兼容问题），直接运行下方 AutoProcessor 流程

In [ ]:
# pipeline 在 Kaggle 易触发 numpy/scipy 冲突，已跳过。直接运行下方「加载原始 MedGemma」即可
pass

# Cell 2: 环境检查（需先运行上方安装）

In [ ]:
import sys
print(f"Python: {sys.version}")

import torch
print(f"PyTorch: {torch.__version__}, CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    USE_BF16 = torch.cuda.get_device_capability(0)[0] >= 8
    DTYPE = torch.bfloat16 if USE_BF16 else torch.float16
    print(f"精度: {'BF16' if USE_BF16 else 'FP16 (P100)'}")
else:
    DTYPE = torch.float32

# Cell 3: 导入与路径配置

In [ ]:
import sys
import os
TF_ENV = "/content/tf_env" if os.path.exists("/content") else "/kaggle/working/tf_env"
for p in ["/kaggle/working/transformers_4.46", "/content/transformers_4.46"]:
    if p in sys.path:
        sys.path.remove(p)
for k in list(sys.modules.keys()):
    if k == "transformers" or k.startswith("transformers."):
        del sys.modules[k]
if TF_ENV not in sys.path:
    sys.path.insert(0, TF_ENV)
elif sys.path[0] != TF_ENV:
    sys.path.remove(TF_ENV)
    sys.path.insert(0, TF_ENV)

import gc
import torch
import pandas as pd
from PIL import Image
from tqdm import tqdm

# Kaggle 路径：Add Input 添加 mimic-cxr-dataset
DATASET_ROOT = "/kaggle/input/mimic-cxr-dataset/official_data_iccv_final"
CSV_CANDIDATES = [
    "/kaggle/input/mimic-cxr-dataset/mimic_eval_single_image_final_233.csv",
    "/kaggle/input/mimic-cxr-dataset/official_data_iccv_final/mimic_eval_single_image_final_233.csv",
    "/kaggle/input/mimic-eval-233/mimic_eval_single_image_final_233.csv",
    "/kaggle/working/mimic_eval_single_image_final_233.csv",
    "./mimic_eval_single_image_final_233.csv",
]
CSV_PATH = next((p for p in CSV_CANDIDATES if os.path.exists(p)), CSV_CANDIDATES[0])
print(f"CSV: {CSV_PATH}")
print(f"Dataset: {DATASET_ROOT}")

# Cell 4: 加载原始 MedGemma（全精度）并测量 GPU 占用

In [ ]:
pass  # 诊断 cell 已移除，已改用 pip 降级方案

In [ ]:
# Kaggle: 使用 TF_ENV 的 transformers，torch 沿用 env check 已加载的
import sys, os
TF_ENV = "/kaggle/working/tf_env"
for p in ["/kaggle/working/transformers_4.46"]:
    if p in sys.path:
        sys.path.remove(p)
for k in list(sys.modules.keys()):
    if k == "transformers" or k.startswith("transformers."):
        del sys.modules[k]
if TF_ENV not in sys.path or sys.path[0] != TF_ENV:
    if TF_ENV in sys.path:
        sys.path.remove(TF_ENV)
    sys.path.insert(0, TF_ENV)

from transformers import AutoProcessor, AutoModelForImageTextToText

model_id = "google/medgemma-1.5-4b-it"
print(f"加载原始 MedGemma ({DTYPE})...")

model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    torch_dtype=DTYPE,
    device_map="auto",
)
processor = AutoProcessor.from_pretrained(model_id)

if torch.cuda.is_available():
    mem_gb = torch.cuda.memory_allocated(0) / (1024**3)
    print(f"原始模型 GPU 占用: {mem_gb:.2f} GB")
    torch.cuda.reset_peak_memory_stats()

# Cell 5: 图像到报告生成

In [ ]:
import ast
import re

PROMPT_TEMPLATE = (
    "You are an expert radiologist. Describe this {view} view chest X-ray. "
    "Provide a concise report consisting of Findings and Impression. "
    "Focus on the heart, lungs, mediastinum, pleural space, and bones. "
    "Do NOT use bullet points, asterisks, or section headers. "
    "Do NOT include disclaimers or 'AI' warnings. Output pure medical text only."
)

def get_single_image_path(cell_val):
    if pd.isna(cell_val): return None
    s = str(cell_val).strip().replace("[","").replace("]","").replace("'","").replace('"',"").split(",")[0].strip()
    if "files" in s: rel = "files" + s.split("files",1)[1]
    else: rel = s.strip("/")
    full = os.path.join(DATASET_ROOT, rel) if not rel.startswith("/") else rel
    return full if os.path.exists(full) else None

def generate_report(model, processor, img_path, view="PA"):
    if not os.path.exists(img_path): return ""
    try: img = Image.open(img_path).convert("RGB")
    except: return ""
    prompt = PROMPT_TEMPLATE.format(view=view)
    msgs = [{"role":"user","content":[{"type":"image","image":img},{"type":"text","text":prompt}]}]
    inp = processor.apply_chat_template(msgs, add_generation_prompt=True, tokenize=True, return_dict=True, return_tensors="pt").to(model.device, dtype=DTYPE)
    L = inp["input_ids"].shape[-1]
    with torch.inference_mode():
        out = model.generate(**inp, max_new_tokens=300, do_sample=False)
    txt = processor.decode(out[0][L:], skip_special_tokens=True)
    return re.sub(r'\s+', ' ', txt.replace("Findings:","").replace("Impression:","")).strip()

df = pd.read_csv(CSV_PATH)
# Ground_Truth 必须用人类报告(text)，绝不能是 prompt
GT_COL = "Ground_Truth" if "Ground_Truth" in df.columns else "text"
IMG_COL = "Image_Path" if "Image_Path" in df.columns else None

rows_out = []
NUM = min(50, len(df))
for idx, row in tqdm(df.head(NUM).iterrows(), total=NUM):
    path, view = None, "PA"
    if IMG_COL:
        path, view = row.get(IMG_COL), row.get("View", "PA")
    else:
        for c, v in [("PA","PA"),("AP","AP"),("Lateral","Lateral")]:
            if c in df.columns and (p := get_single_image_path(row.get(c))):
                path, view = p, v
                break
    if not path: continue
    gt = str(row.get(GT_COL) or "").strip()
    if not gt or gt.startswith("You are"): continue  # 防止 prompt 当 Ground_Truth
    rep = generate_report(model, processor, path, view)
    rows_out.append({"subject_id":row["subject_id"],"View":view,"Image_Path":path,"Ground_Truth":gt,"Generated_Report":rep})

df_sub = pd.DataFrame(rows_out)
print(f"生成 {len(df_sub)} 条")

# Cell 6: RadGraph F1 评估

In [ ]:
from radgraph import F1RadGraph
import numpy as np

refs = df_sub["Ground_Truth"].fillna("").tolist()
hyps = df_sub["Generated_Report"].fillna("").tolist()

f1radgraph = F1RadGraph(reward_level="all", model_type="modern-radgraph-xl")
mean_reward, reward_list, _, _ = f1radgraph(hyps=hyps, refs=refs)
rg_e, rg_er, rg_er_bar = mean_reward

print("=" * 50)
print("原始 MedGemma (W4A16/FP16) RadGraph F1 分数")
print("-" * 50)
print(f"RG_E (仅实体):        {float(rg_e)*100:.2f}")
print(f"RG_ER (实体+关系):   {float(rg_er)*100:.2f}  <- 论文常用")
print(f"RG_ER_bar (完整):    {float(rg_er_bar)*100:.2f}")
print("=" * 50)

ORIGINAL_SCORES = {"rg_e": float(rg_e), "rg_er": float(rg_er), "rg_er_bar": float(rg_er_bar)}
ORIGINAL_GPU_GB = torch.cuda.max_memory_allocated(0) / (1024**3) if torch.cuda.is_available() else 0
print(f"\n原始模型峰值 GPU: {ORIGINAL_GPU_GB:.2f} GB")

# Cell 7: 释放 GPU 并保存结果（供后续对比）

In [ ]:
del model
del processor
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
print("已释放原始模型，GPU 可加载量化模型")

# 保存原始模型生成结果，供 W4A4/W4A8 Notebook 对比
df_sub.to_csv("/kaggle/working/original_medgemma_results.csv", index=False)
import json
with open("/kaggle/working/original_scores.json", "w") as f:
    json.dump({"scores": ORIGINAL_SCORES, "gpu_gb": ORIGINAL_GPU_GB}, f)
print("结果已保存至 /kaggle/working/")

# Cell 8: 与量化模型对比说明

本 Notebook 为**基线**。运行 W4A4、W4A8 Notebook 后，将得到：
- 原始: RG_ER ≈ 27–30（MIMIC-CXR 论文值）
- W4A4: 预期略降，显存显著减少
- W4A8: 预期接近原始，显存减少

**重要**：每次只加载一个模型，跑完删除再加载下一个，才能准确对比 GPU 占用。